# AI vs. Human
Assuming the same image segmenter and models, results are predetermined for labeling.
Thus, we can easily compare on an image-to-image basis the assigned Region and its ML-assigned label ("Labels") and compare against the human assigned labels ("Labels_RC" for Rachel, can change based on person)

In [1]:
# Imports
import sys
sys.path.append("..")
from Utility.segmentation_utils import *
import Utility.model_utils as mu
import matplotlib.pyplot as plt
import os

In [2]:
### DEFINE VARIABLES OF INTEREST HERE ###
# vvv Three key values to change vvv

## Define folder path of interest, image of interest, and csv path for folder of interest
# Note: Can maybe automate this, but name changing makes it difficult
img_folder = "../Images/Organized images 20230613/L-3_nM-10_au10_mixing-T_oven-T_embed-Ag"
image_name = "L-3_nM-3_au10_mixing-T_oven-T_embed-Ag_01.tif"
csv_path = "../Results/Organized images 20230613/L-3_nM-10_au10_mixing-T_oven-T_embed-Ag_edge-darkbright_thresh-ensemble.csv"

# Runtime settings to compare against
edge = "darkbright"
thresh = "ensemble"

# Derive variables
img_path = os.path.join(img_folder,image_name)

# Check that the files exist
assert(os.path.exists(img_folder))
assert(os.path.exists(img_path))
assert(os.path.exists(csv_path))

AssertionError: 

In [ ]:
# Get the already processed csv
